<a href="https://colab.research.google.com/github/takky0330/wisper/blob/main/wisper_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install moviepy pyannote-audio

In [ ]:
import numpy as np
from pyannote.audio import Pipeline, Audio
from pyannote.core import Segment
from pyannote.audio.pipelines.speaker_verification import PretrainedSpeakerEmbedding

In [ ]:
!pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip

In [ ]:
video_path = "./9d4fc932d442ef2ffb17d0a8086390e6_1725887318814_0.mp4"
audio_path = '.'.join(video_path.split('.')[:-1]) + '.wav'
video = VideoFileClip(video_path)
video.audio.write_audiofile(audio_path, ffmpeg_params=["-ac", "1"])  #モノラルで出力

In [ ]:
!pip install git+https://github.com/openai/whisper.git

In [ ]:
import whisper
import json, re

In [ ]:
model = whisper.load_model("small")

In [ ]:
result = model.transcribe(audio_path, verbose=True, fp16=False, language="ja")

In [ ]:
segments = result['segments']
prog = re.compile('教えて|聞か|聞き|願い|ください|ですか|でしょうか|ましたか|されましたか|いつ頃')
INTERVIEW = []
for seg in segments:
  count = len(prog.findall(seg['text']))
  if count > 0: print(seg['text'])
  _seg = {'start': seg['start'], 'end': seg['end'], 'text': seg['text'], 'count': count}
  INTERVIEW.append(_seg)

In [ ]:
segments = result['segments']
embedding_model = PretrainedSpeakerEmbedding("speechbrain/spkrec-ecapa-voxceleb", device='cpu')
embeddings = np.zeros(shape=(len(segments), 192))

duration = video.duration
embeddings = []


for i, segment in enumerate(segments):
    audio = Audio()
    start = segment["start"]
    end = min(duration, segment["end"])
    clip = Segment(start, end)
    waveform, sample_rate = audio.crop(audio_path, clip)
    print(waveform[0])
    _enbed = embedding_model(waveform[None])
    #print(_enbed[0])
    embeddings.append(_enbed[0])

embeddings = np.nan_to_num(embeddings)


In [ ]:
from sklearn.cluster import AgglomerativeClustering
speaker_count = 2

In [ ]:
len(embeddings)
embeddings.shape

(96, 192)

In [ ]:
clustering = AgglomerativeClustering(speaker_count).fit(embeddings)

In [ ]:
labeled_segments = []
for label, segment in zip(clustering.labels_, result["segments"]):
    labeled_segments.append((label, segment["start"], segment["text"]))

interivew = []
print(INTERVIEW)
for s, (speaker, _, text) in enumerate(labeled_segments):
    stime = INTERVIEW[s]["start"]
    interivew.append(f"話者{speaker + 1} :  {stime} 〜 : 「{text}」")

In [ ]:
for speek in interivew:
  print(speek)